In [2]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [21]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
    #add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
sets = [
'9f86422b-08a3-4926-9ff8-22f8a0be1c48',
'9135be43-33c6-463d-87c7-9e2b1d56adbd',
'10d1c941-87af-44d9-ac29-8bdcb15d4ca7',
'e58e2141-9253-4c91-85c3-d67ce06db28f',
'aa6f0d82-16c9-4109-a1f8-3fa0636a3560',
'e6ce55ac-a39f-4064-a578-5329c5454205']
    
my_exp = sets[5]
    
store_dict = {} 
item_uuids = []

store, uuids = record_object(my_exp, store_dict, item_uuids)

print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
analysis_step f7b8cd87-49e5-4c33-afab-9ec90d65faf3 released to project
file_processed cd0aa2ca-a790-48b7-9923-3240354b71cc released to project
file_processed 925fd3f1-4fe4-4338-8892-6778c603ee7e released to project
file_processed d7658036-b5c0-463c-9a12-2e5f7eb006a9 released to project
file_processed 694c29c9-cbcd-4581-88d8-44304b9d4d6e released to project
workflow_run_sbg 06555d7d-4b52-47e6-a8c1-1915fac6bcf2 released to project
workflow_run_sbg 57655534-fd8b-4dc4-accc-602498dbba74 released to project
463
463


In [23]:
## This part is only for releasing
counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print 'deleted data', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released to project"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)
print counter

file_processed uploaded 32a5b9a9-2e4c-44d2-8d71-5839a43719dc
file_processed uploaded 0cdfaadd-f90d-4f48-bf40-34af22e2d21b
file_processed uploaded 027702d7-30be-4825-949c-ed95220b1458
file_processed uploaded 2051ec0b-fbb3-4eb1-a1f7-2484ae908fea
file_processed uploaded 87e00d94-714e-4ace-883e-6b13ae525df0
file_processed uploaded 08906114-4018-4705-a87c-fcebb716e17e
file_processed uploaded 5cf59963-335e-498d-8776-e85cfa22feb6
file_processed uploaded e1a1b045-8641-4c87-ae6e-dfc756928486
file_processed uploaded 95ec31d4-9950-49ea-b4e3-ca214f0ee518
file_processed uploaded 0d3fd098-f8d3-469d-9489-0b50a4875325
file_processed uploaded 39c45ccc-957b-4743-8c1c-865d2753575a
file_processed uploaded 7f7c6546-4190-4ac1-81d1-29c3309ebfb3
file_processed uploaded efe548c6-3cee-4b8e-b180-141b9631a74e
file_processed uploaded a9ecf94f-2fcb-4009-8ad6-26084c01e3d2
file_processed uploaded 124dba40-1aff-4867-9667-9001e2c50e6c
file_processed uploaded 4aff7b17-d834-406b-8d7a-243ff33487d7
file_processed uploaded 

workflow_run_sbg in review by lab cf98a82b-921c-492b-a022-372a9e5d00a4
workflow_run_sbg in review by lab c742ecf6-b573-4a89-82ef-9a9ef253c462
workflow_run_sbg in review by lab 609ec60e-de86-4342-8a76-d37561f68c93
workflow_run_sbg in review by lab 3cdb5f45-968f-48da-8f08-56af2847a0c7
workflow_run_sbg in review by lab d15c552a-f3ea-4cb8-828a-33419ebc7e38
workflow_run_sbg in review by lab 39fe6a13-3bb0-49f2-901e-4858439958c9
workflow_run_sbg in review by lab e8b8390b-e4ea-4911-86ff-e01e2dd314fa
workflow_run_sbg in review by lab e6a04651-13e8-4bc3-b670-d1aa8092282c
workflow_run_sbg in review by lab 6d5074e2-c2de-4f39-bb69-954594cde0d6
workflow_run_sbg in review by lab 53843556-cbc4-43a0-91af-f7e535bf54ee
workflow_run_sbg in review by lab 1a396b3d-1ec4-4a6d-9622-36cc770db0d7
workflow_run_sbg in review by lab d179938d-a2d0-418c-9d7d-45ac67467f76
workflow_run_sbg in review by lab 23f81430-1ded-47fc-8fe2-f86fe3d623a4
workflow_run_sbg in review by lab 6caa397b-31a7-4ada-ba7b-dee382001473
workfl

workflow_run_sbg in review by lab 347a028e-ab34-460e-b139-f3a227ce8d07
workflow_run_sbg in review by lab d2351bda-0e78-49de-a3a3-e65441ebdd38
workflow_run_sbg in review by lab 8e762af4-a971-42bf-91ff-86951d7682c4
workflow_run_sbg in review by lab c4ff0d76-c7af-4634-a1d1-a30ed2fb0284
workflow_run_sbg in review by lab 0683c548-51e7-42b1-a28b-833cbfae093d
workflow_run_sbg in review by lab 451a2f8a-a168-4d5d-a02d-21402a2e854f
workflow_run_sbg in review by lab 2519dc95-3f8d-4437-9372-226d639fa801
workflow_run_sbg in review by lab dbb9d6aa-f86f-4701-968f-35301083431d
workflow_run_sbg in review by lab 9aa33fea-1abc-4bc6-9280-d06d91cff50f
workflow_run_sbg in review by lab 0b240980-21be-4b96-b67e-0ac10c728973
workflow_run_sbg in review by lab 539bb87b-094b-4595-b7de-5274a6ced1dd
workflow_run_sbg in review by lab 7a518168-72e2-47a6-adb9-8bab594efe8d
workflow_run_sbg in review by lab 4e4b48ba-cd69-4154-aaf2-8d7ad5c8f838
workflow_run_sbg in review by lab 71e8c0d0-32ec-49d9-ae94-63c56dd9629d
workfl